# ChronoTick 2: Zero-Shot TSFM Benchmark

Benchmark all 5 time series foundation models on clock drift prediction
across 4 machines, with and without covariates, at multiple context lengths
and horizons.

## Models Tested
| Model | Params | Covariates | Quantiles |
|-------|--------|------------|----------|
| Chronos-2 (Small) | 28M | Yes | Yes |
| Chronos-2 (Base) | 120M | Yes | Yes |
| TimesFM 2.5 | 200M | Yes (XReg) | Yes |
| Moirai 1.1 (Small) | 14M | Yes | Yes |
| Toto | 151M | Yes | Yes (samples) |
| Granite TTM | 1-5M | No (zero-shot) | No |

## Setup

**Colab (recommended):** Use a T4 GPU runtime. Upload to `My Drive/chronotick2/`:
1. `tick2/` -- the Python package (this repo's `tick2/` directory)
2. `data/24h_snapshot/` -- the sensor CSVs (from `sensors/data/24h_snapshot/`)

**Local:** Run from `tick2/` directory. Install model deps individually.

In [ ]:
# === Environment Detection & Setup ===
import os
import sys

IN_COLAB = "COLAB_GPU" in os.environ or os.path.exists("/content")

if IN_COLAB:
    from google.colab import drive
    drive.mount("/content/drive")
    DATA_DIR = "/content/drive/MyDrive/chronotick2/data"
    RESULTS_DIR = "/content/drive/MyDrive/chronotick2/results"
else:
    DATA_DIR = None  # uses default
    RESULTS_DIR = "../results"

In [ ]:
# === Install Dependencies (Colab) ===
# Run ONE model at a time to avoid dependency conflicts.
# Uncomment the model you want to benchmark.

if IN_COLAB:
    # Install tick2 from Drive
    !pip install -q /content/drive/MyDrive/chronotick2/tick2/

    # --- Uncomment ONE model group at a time ---

    # Chronos-2 (Amazon)
    # !pip install -q "chronos-forecasting[extras]>=2.2"

    # Moirai (Salesforce)
    # !pip install -q "uni2ts>=2.0"

    # Toto (Datadog)
    # !pip install -q toto-ts

    # Granite TTM (IBM)
    # !pip install -q "granite-tsfm>=0.3.3"

    # TimesFM 2.5 (Google) - requires source install
    # !git clone https://github.com/google-research/timesfm /content/timesfm
    # !cd /content/timesfm && pip install -q -e ".[torch]"

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch

from tick2.benchmark.metrics import mae, rmse, coverage
from tick2.benchmark.reporting import format_summary, results_to_latex, save_results
from tick2.benchmark.runner import BenchmarkConfig, results_to_dataframe, run_benchmark
from tick2.benchmark.timing import get_gpu_memory_mb, profile_inference, reset_gpu_memory_stats
from tick2.data.preprocessing import TARGET_COL, get_feature_cols, load_all
from tick2.models.registry import get_model, list_models
from tick2.utils.gpu import clear_gpu_memory

sns.set_theme(style="whitegrid", font_scale=1.1)

# GPU info
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_mem / 1024**3:.1f} GB")
else:
    print("Running on CPU")

print(f"Available models: {list_models()}")

## 1. Load Data

In [ ]:
data_dir = Path(DATA_DIR) if DATA_DIR else None
datasets = load_all(data_dir=data_dir, snapshot="24h_snapshot")
print(f"\nLoaded {len(datasets)} machines")

## 2. Benchmark Configuration

In [ ]:
config = BenchmarkConfig(
    context_lengths=[512, 1024],
    horizons=[60, 120],
    n_samples=25,
    seed=42,
    use_covariates=[False, True],
    quantile_alpha=0.2,
)

# Select which models to benchmark
# Change this list based on which deps are installed
MODELS_TO_RUN = [
    # "chronos2-small",
    # "chronos2-base",
    # "timesfm-2.5",
    # "moirai-1.1-small",
    # "toto",
    # "granite-ttm",
]

if not MODELS_TO_RUN:
    print("WARNING: No models selected! Uncomment at least one model above.")
    print("Also make sure the corresponding pip package is installed.")

## 3. Run Benchmarks

Each model is loaded, benchmarked, then unloaded to free GPU memory.
On a T4, expect ~20-40 minutes per model depending on the config.

In [ ]:
all_results = []

for model_name in MODELS_TO_RUN:
    print(f"\n{'='*60}")
    print(f"Benchmarking: {model_name}")
    print(f"{'='*60}")

    clear_gpu_memory()
    reset_gpu_memory_stats()

    try:
        model = get_model(model_name)
        model.load(device=device)
        print(f"  Loaded on {device}, ~{model.memory_footprint_mb():.0f} MB")

        results = run_benchmark(model, datasets, config, progress=True)
        all_results.extend(results)

        print(f"  Completed: {len(results)} configurations")

    except Exception as e:
        print(f"  FAILED: {e}")
        import traceback
        traceback.print_exc()

    finally:
        # Free model memory
        del model
        clear_gpu_memory()

print(f"\nTotal results: {len(all_results)} configurations")

## 4. Results Summary

In [ ]:
if all_results:
    results_df = results_to_dataframe(all_results)
    print(format_summary(results_df))
    display(results_df)
else:
    print("No results to display. Select and run models in section 2.")

## 5. Visualization

In [ ]:
if all_results:
    df = results_df.copy()

    # --- MAE by Model and Machine ---
    fig, ax = plt.subplots(figsize=(12, 5))
    uni_df = df[~df["with_covariates"]]
    if not uni_df.empty:
        pivot = uni_df.pivot_table(values="mae", index="model", columns="machine")
        pivot.plot(kind="bar", ax=ax)
        ax.set_ylabel("MAE (ppm)")
        ax.set_title("Zero-Shot MAE by Model and Machine (Univariate)")
        ax.legend(title="Machine")
        plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # --- Covariate Effect ---
    cov_models = df[df["model"].isin(df[df["with_covariates"]]["model"].unique())]
    if not cov_models.empty and len(cov_models["with_covariates"].unique()) > 1:
        fig, ax = plt.subplots(figsize=(10, 5))
        sns.barplot(data=cov_models, x="model", y="mae", hue="with_covariates", ax=ax)
        ax.set_ylabel("MAE (ppm)")
        ax.set_title("Covariate Effect: Univariate vs. Multivariate")
        ax.legend(title="With Covariates")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

    # --- Inference Time ---
    fig, ax = plt.subplots(figsize=(10, 5))
    time_data = df.groupby("model")["inference_ms"].mean().sort_values()
    time_data.plot(kind="barh", ax=ax, color="steelblue")
    ax.set_xlabel("Mean Inference Time (ms)")
    ax.set_title("Inference Latency by Model")
    plt.tight_layout()
    plt.show()

## 6. Context Length Sensitivity

In [ ]:
if all_results:
    uni_df = df[~df["with_covariates"]]
    if len(uni_df["context_length"].unique()) > 1:
        fig, ax = plt.subplots(figsize=(10, 5))
        sns.lineplot(
            data=uni_df,
            x="context_length",
            y="mae",
            hue="model",
            style="model",
            markers=True,
            ax=ax,
        )
        ax.set_xlabel("Context Length (timesteps)")
        ax.set_ylabel("MAE (ppm)")
        ax.set_title("MAE vs. Context Length")
        plt.tight_layout()
        plt.show()

## 7. Save Results

In [ ]:
if all_results:
    output_dir = Path(RESULTS_DIR)
    csv_path, latex_path = save_results(results_df, output_dir, prefix="zero_shot")
    print(f"CSV saved: {csv_path}")
    print(f"LaTeX saved: {latex_path}")

    # Print LaTeX table for copy-paste into paper
    print("\n" + results_to_latex(results_df))

## 8. Per-Model Timing Profile (Optional)

Detailed timing with warm-up runs for publication-quality latency numbers.

In [ ]:
# Uncomment to run detailed timing for a specific model
# This takes ~2-5 minutes per model.

# TIMING_MODEL = "chronos2-small"
# clear_gpu_memory()
# model = get_model(TIMING_MODEL)
# model.load(device=device)
#
# first_machine = list(datasets.keys())[0]
# first_df = datasets[first_machine][0]
# context = first_df[TARGET_COL].values[:1024]
#
# for hz in [60, 120]:
#     timing = profile_inference(model, context, horizon=hz, n_warmup=5, n_repeats=20)
#     print(f"{TIMING_MODEL} hz={hz}: {timing.mean_ms:.1f} +/- {timing.std_ms:.1f} ms "
#           f"(median={timing.median_ms:.1f}, p95={timing.p95_ms:.1f})")
#
# del model
# clear_gpu_memory()